In [1]:
import socket
import threading
import subprocess
import os
from dotenv import load_dotenv



In [ ]:
tx_lock = threading.Lock()
SERVER_IP = os.getenv("LISTEN_IP")
SERVER_PORT = int(os.getenv("SERVER_PORT"))

def handle_client(conn, addr):
    print(f"New connection from {addr}")
    conn.sendall("Continue".encode())
    total_len = 0

    # TODO: Check if file already exists, if it does, remove it
    # No need to do this, b+w already flushes out file

    # Connection closes after writing to file
    with open("../startup/test.raw", "b+w") as f: 
        with conn:
            while True:
                data = conn.recv(1024)
                if not data:
                    print(f"Connection closed by {addr}")
                    break
                f.write(data)
                total_len += len(data)
            
    # Handle running signal and deleting file
    # TODO: Run subprocess
    # subprocess.run(["./transmit.elf", "--tx-file=test.raw", "--tx-gain=7"])
    # subprocess.run(["python", "../helloworld.py"])
    # TODO: Remove file
    # subprocess.run(["rm", "./startup/data.raw"])

    print("Done")
    tx_lock.release_lock()
    
def start_server():
    # Set up server
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server_socket.bind((SERVER_IP, SERVER_PORT))
    server_socket.listen(5)

    print(f"Server listening on {SERVER_IP}:{SERVER_PORT}")

    while True:
        print("Waiting for a new connection...")
        conn, addr = server_socket.accept()

        # Only handle one client at a time
        if tx_lock.acquire_lock(blocking=False):
            
            # Punt client onto a new thread
            thread = threading.Thread(target=handle_client, args=(conn, addr))
            thread.start()

            print("Ready for the next client.")
        else: 
            conn.sendall("Failed to run".encode())
            conn.close()

if __name__ == "__main__":
    start_server()


TypeError: gethostbyaddr() takes exactly 1 argument (0 given)